## Echoes Of Chaos

In [135]:
SCOPETYPE = 'CWNANO'
PLATFORM = 'CWNANO'

In [136]:
%run "../setup/Setup_Generic.ipynb"

(ChipWhisperer NAEUSB WARNING|File naeusb.py:826) Your firmware (0.65.0) is outdated - latest is 0.66.0 See https://chipwhisperer.readthedocs.io/en/latest/firmware.html for more information


INFO: Caught exception on reconnecting to target - attempting to reconnect to scope first.
INFO: This is a work-around when USB has died without Python knowing. Ignore errors above this line.
INFO: Found ChipWhisperer😍


In [137]:
cw.program_target(scope, prog, "chaos-{}.hex".format(PLATFORM))

Detected known STMF32: STM32F04xxx
Extended erase (0x44), this can take ten seconds or more
Attempting to program 14307 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 14307 bytes


In [113]:
import struct, time
import numpy as np
import chipwhisperer as cw

# try to import torch
try:
    import torch
    import torch.nn as nn
    import torch.optim as optim

# ----------------------------
# config
# ----------------------------
ARR_LEN = 15
TRACE_KEEP = 3000
N_LEVELS = 8
SUBDIV = 4
LOCAL_MARGIN = 4        # DL will look at [lo-4 .. hi+4]
DL_EPOCHS = 50
DL_LR = 1e-2

scope = cw.scope()
target = cw.target(scope, cw.targets.SimpleSerial)
scope.adc.samples = 100000

trace_cache = {}

# ----------------------------
# device helpers
# ----------------------------
def reset_ss():
    target.simpleserial_write('x', b"")
    _ = target.simpleserial_read('r', 1)

def num_q():
    target.simpleserial_write('q', b"0")
    resp = target.simpleserial_read('r', 4)
    return struct.unpack("<I", resp)[0]

def capture_val_skip(v, skip):
    low, high = struct.pack('<H', v)
    pkt = [0, low, high, skip]
    scope.arm()
    target.simpleserial_write('p', bytes(pkt))
    timeout = scope.capture()
    if timeout:
        raise RuntimeError("capture timeout")
    _ = target.simpleserial_read('r', 1)
    return scope.get_last_trace()[:TRACE_KEEP].copy()

def get_trace(v, skip):
    key = (v, skip)
    if key not in trace_cache:
        reset_ss()
        trace_cache[key] = capture_val_skip(v, skip)
    return trace_cache[key]

def check_arr_exact_15(vals):
    vals = sorted(vals)
    bs = []
    for v in vals:
        lo, hi = struct.pack('<H', v)
        bs.append(lo); bs.append(hi)
    scope.arm()
    target.simpleserial_write('a', bytes(bs))
    timeout = scope.capture()
    if timeout:
        raise RuntimeError("timeout in 'a'")
    resp = target.simpleserial_read('r', 20)
    _ = scope.get_last_trace()
    return resp.decode(errors="ignore")

# ----------------------------
# projector per skip
# ----------------------------
def build_discriminant_for_skip(skip):
    # avg 2 shots
    t_small = 0.5 * (get_trace(0, skip).astype(np.float32) +
                     get_trace(0, skip).astype(np.float32))
    t_big   = 0.5 * (get_trace(65535, skip).astype(np.float32) +
                     get_trace(65535, skip).astype(np.float32))

    w = t_small - t_big
    s_small = float(np.dot(t_small, w))
    s_big   = float(np.dot(t_big,   w))
    thresh  = 0.5 * (s_small + s_big)
    return w, thresh

def score_trace(tr, w):
    return float(np.dot(tr, w))

# ----------------------------
# tiny local DL model
# ----------------------------
class Tiny1DCNN(nn.Module):
    def __init__(self, in_len):
        super().__init__()
        self.conv1 = nn.Conv1d(1, 8, kernel_size=9, padding=4)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool1d(2)
        self.conv2 = nn.Conv1d(8, 16, kernel_size=7, padding=3)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool1d(2)
        L = in_len // 2
        L = L // 2
        self.fc1 = nn.Linear(16 * L, 32)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(32, 1)

    def forward(self, x):
        x = self.conv1(x); x = self.relu1(x); x = self.pool1(x)
        x = self.conv2(x); x = self.relu2(x); x = self.pool2(x)
        x = x.flatten(1)
        x = self.fc1(x); x = self.relu3(x)
        x = self.fc2(x)
        return x

def deep_refine(skip, w, thresh, lo, hi):

    win_lo = max(0, lo - LOCAL_MARGIN)
    win_hi = min(0xFFFF, hi + LOCAL_MARGIN)
    cand_vals = list(range(win_lo, win_hi + 1))

    traces = []
    labels = []
    for v in cand_vals:
        tr = get_trace(v, skip).astype(np.float32)
        tr = (tr - tr.mean()) / (tr.std() + 1e-6)
        traces.append(tr)
        sc = score_trace(tr, w)
        lab = 1 if sc < thresh else 0   # BIG=1
        labels.append(lab)

    traces = np.stack(traces, axis=0)
    labels = np.array(labels, dtype=np.float32)[:, None]

    L = traces.shape[1]
    model = Tiny1DCNN(L)
    crit = nn.BCEWithLogitsLoss()
    opt = optim.Adam(model.parameters(), lr=DL_LR)

    x_t = torch.from_numpy(traces).unsqueeze(1)
    y_t = torch.from_numpy(labels)

    model.train()
    for _ in range(DL_EPOCHS):
        opt.zero_grad()
        out = model(x_t)
        loss = crit(out, y_t)
        loss.backward()
        opt.step()

    model.eval()
    with torch.no_grad():
        logits = model(x_t).squeeze(1).numpy()
        probs = 1 / (1 + np.exp(-logits))
    big_idxs = [i for i, p in enumerate(probs) if p > 0.5]
    if not big_idxs:
        return max(0, hi - 1)
    best_idx = min(big_idxs)
    return cand_vals[best_idx]

# ----------------------------
# multiresolution
# ----------------------------
def find_value_for_skip(skip):
    w, thresh = build_discriminant_for_skip(skip)
    active = [(0, 65535, True, False)]
    for _ in range(N_LEVELS):
        new_active = []
        for (lo, hi, lo_small, hi_small) in active:
            if lo_small == hi_small:
                continue
            span = hi - lo
            if span <= 1:
                new_active.append((lo, hi, lo_small, hi_small))
                continue
            step = max(span // SUBDIV, 1)
            points = [lo + k * step for k in range(1, SUBDIV)]
            if points and points[-1] >= hi:
                points = points[:-1]
            samples = [(lo, lo_small)]
            for p in points:
                tr_p = get_trace(p, skip)
                sc_p = score_trace(tr_p, w)
                cls_p = (sc_p >= thresh)
                samples.append((p, cls_p))
            samples.append((hi, hi_small))
            for i in range(len(samples)-1):
                x0, c0 = samples[i]
                x1, c1 = samples[i+1]
                if c0 != c1:
                    new_active.append((x0, x1, c0, c1))
        active = new_active
        if not active:
            break
    if not active:
        return 0
    best_seg = min(active, key=lambda s: s[1])
    lo, hi, _, _ = best_seg
    val = deep_refine(skip, w, thresh, lo, hi)
    return val

# ----------------------------
# main
# ----------------------------
reset_ss()
start_q = num_q()

recovered = []
for skip in range(ARR_LEN):
    v = find_value_for_skip(skip)
    recovered.append(v)
    print(f"[+] index {skip} -> {v}")

print("[*] recovered (unsorted):", recovered)

recovered = [max(0, v - 1) for v in recovered]

recovered_sorted = sorted(recovered)
print("[*] recovered (bias-fixed, sorted):", recovered_sorted)

flag = check_arr_exact_15(recovered_sorted)
end_q = num_q()
print("[*] FLAG:", flag)
print("[*] queries used:", end_q - start_q)


(ChipWhisperer NAEUSB WARNING|File naeusb.py:826) Your firmware (0.65.0) is outdated - latest is 0.66.0 See https://chipwhisperer.readthedocs.io/en/latest/firmware.html for more information
/tmp/ipykernel_263281/339471976.py:161: RuntimeWarning: overflow encountered in exp
  probs = 1 / (1 + np.exp(-logits))


[+] index 0 -> 27965
[+] index 1 -> 698
[+] index 2 -> 45997
[+] index 3 -> 36081
[+] index 4 -> 58773
[+] index 5 -> 9093
[+] index 6 -> 42350
[+] index 7 -> 6537
[+] index 8 -> 13984
[+] index 9 -> 25987
[+] index 10 -> 64749
[+] index 11 -> 64987
[+] index 12 -> 29601
[+] index 13 -> 49205
[+] index 14 -> 19618
[*] recovered (unsorted): [27965, 698, 45997, 36081, 58773, 9093, 42350, 6537, 13984, 25987, 64749, 64987, 29601, 49205, 19618]
[*] recovered (bias-fixed, sorted): [697, 6536, 9092, 13983, 19617, 25986, 27964, 29600, 36080, 42349, 45996, 49204, 58772, 64748, 64986]
[*] FLAG: eoc{th3yreC00ked}   
[*] queries used: 466


In [138]:
import struct, time
import numpy as np
import chipwhisperer as cw
from sklearn.linear_model import SGDClassifier

# ----------------------------
# config
# ----------------------------
ARR_LEN = 15
TRACE_KEEP = 3000
MAX_VAL = 0xFFFF

scope = cw.scope()
target = cw.target(scope, cw.targets.SimpleSerial)
scope.adc.samples = 100000

trace_cache = {}

# ----------------------------
# device helpers
# ----------------------------
def reset_ss():
    target.simpleserial_write('x', b"")
    _ = target.simpleserial_read('r', 1)

def num_q():
    target.simpleserial_write('q', b"0")
    resp = target.simpleserial_read('r', 4)
    return struct.unpack("<I", resp)[0]

def capture_val_skip(v, skip):
    low, high = struct.pack('<H', v)
    pkt = [0, low, high, skip]
    scope.arm()
    target.simpleserial_write('p', bytes(pkt))
    timeout = scope.capture()
    if timeout:
        raise RuntimeError("capture timeout")
    _ = target.simpleserial_read('r', 1)
    return scope.get_last_trace()[:TRACE_KEEP].copy()

def get_trace(v, skip):
    key = (v, skip)
    if key not in trace_cache:
        reset_ss()
        trace_cache[key] = capture_val_skip(v, skip)
    return trace_cache[key]

def check_arr_exact_15(vals):
    vals = sorted(vals)
    bs = []
    for v in vals:
        lo, hi = struct.pack('<H', v)
        bs.append(lo); bs.append(hi)
    scope.arm()
    target.simpleserial_write('a', bytes(bs))
    timeout = scope.capture()
    if timeout:
        raise RuntimeError("timeout in 'a'")
    resp = target.simpleserial_read('r', 20)
    _ = scope.get_last_trace()
    return resp.decode(errors="ignore")

# ----------------------------
# per-skip ML+binary-search
# ----------------------------
def recover_one(skip):
    # 1) anchor examples: 0 = SMALL, 65535 = BIG
    tr_small = get_trace(0, skip)
    tr_big   = get_trace(65535, skip)
    X = np.vstack([tr_small, tr_big])
    y = np.array([1, 0], dtype=np.int32)   # 1=SMALL, 0=BIG

    # linear classifier; we'll also update as we go
    clf = SGDClassifier(loss="log_loss", max_iter=5, tol=None)
    clf.fit(X, y)

    lo, hi = 0, MAX_VAL
    # classic lower_bound on "is BIG?"
    while lo < hi:
        mid = (lo + hi) // 2
        tr_mid = get_trace(mid, skip)
        # predict
        pred = clf.predict(tr_mid.reshape(1, -1))[0]   # 1=SMALL, 0=BIG
        # optional: on-the-fly adaptation
        clf.partial_fit(tr_mid.reshape(1, -1), np.array([pred]))
        if pred == 1:  # SMALL → go right
            lo = mid + 1
        else:          # BIG → go left
            hi = mid
    cand = lo

    # one-step local correction (this board tends to land on upper side)
    if cand > 0:
        tr_prev = get_trace(cand - 1, skip)
        pred_prev = clf.predict(tr_prev.reshape(1, -1))[0]
        if pred_prev == 0:  # also BIG
            cand = cand - 1

    return cand

# ----------------------------
# main
# ----------------------------
reset_ss()
start_queries = num_q()

recovered = []
for skip in range(ARR_LEN):
    v = recover_one(skip)
    recovered.append(v)
    print(f"[+] index {skip} -> {v}")

print("[*] recovered (unsorted):", recovered)

# your required bias-fix
recovered = [max(0, v - 1) for v in recovered]

recovered_sorted = sorted(recovered)
print("[*] recovered (bias-fixed, sorted):", recovered_sorted)

flag = check_arr_exact_15(recovered_sorted)
end_queries = num_q()

print("[*] FLAG:", flag)
print("[*] queries used:", end_queries - start_queries)


(ChipWhisperer NAEUSB WARNING|File naeusb.py:826) Your firmware (0.65.0) is outdated - latest is 0.66.0 See https://chipwhisperer.readthedocs.io/en/latest/firmware.html for more information


[+] index 0 -> 27965
[+] index 1 -> 698
[+] index 2 -> 45997
[+] index 3 -> 36081
[+] index 4 -> 58773
[+] index 5 -> 9093
[+] index 6 -> 42350
[+] index 7 -> 6537
[+] index 8 -> 13984
[+] index 9 -> 25987
[+] index 10 -> 64749
[+] index 11 -> 64987
[+] index 12 -> 29601
[+] index 13 -> 49205
[+] index 14 -> 19618
[*] recovered (unsorted): [27965, 698, 45997, 36081, 58773, 9093, 42350, 6537, 13984, 25987, 64749, 64987, 29601, 49205, 19618]
[*] recovered (bias-fixed, sorted): [697, 6536, 9092, 13983, 19617, 25986, 27964, 29600, 36080, 42349, 45996, 49204, 58772, 64748, 64986]
[*] FLAG: eoc{th3yreC00ked}   
[*] queries used: 271
